In [ ]:
# Importar librerías
from surprise import Dataset, Reader, KNNBasic, SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

# Cargar el dataset MovieLens
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file('datasets/ml-100k/u.data', reader=reader)

# Dividir el dataset en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Modelo 1: UserKNN (Filtrado Colaborativo Basado en Vecinos)
sim_options = {
    'name': 'cosine',       # Tipo de similitud
    'user_based': True      # Si es False, usa item-item
}

model_knn = KNNBasic(sim_options=sim_options)
# Entrenar el modelo KNN
model_knn.fit(trainset)

# Evaluar el modelo KNN
pred_knn = model_knn.test(testset)

print("KNN - RMSE:")         # Error cuadrático medio
accuracy.rmse(pred_knn)

print("KNN - MAE:")          # Error absoluto medio
accuracy.mae(pred_knn)

In [ ]:
#  Modelo 2: SVD (Factorización de Matrices)

model_svd = SVD()
model_svd.fit(trainset)

pred_svd = model_svd.test(testset)

print("SVD - RMSE:")        # Error cuadrático medio
accuracy.rmse(pred_svd)

print("SVD - MAE:")         # Error absoluto medio
accuracy.mae(pred_svd)

| Modelo  | RMSE   | MAE    | Tiempo |
| ------- | ------ | ------ | ------ |
| UserKNN | 1.0194 | 0.8038 | 2.1s   |
| SVD     | 0.9354 | 0.7376 | 0.8s   |

RMSE (Root Mean Squared Error): Penaliza mucho los errores grandes. Mide cuánto se desvía la predicción del rating real. Más baja = mejor.

MAE (Mean Absolute Error): Mide el error promedio absoluto. Menos sensible a outliers que RMSE. Más baja = mejor.

---

## Análisis: **UserKNN**

| Aspecto          | Análisis                                                                                                                             |
| ---------------- | ------------------------------------------------------------------------------------------------------------------------------------ |
| **RMSE: 1.0194** | Es un valor aceptable, pero más alto que SVD. El modelo no predice tan bien como SVD.                                            |
| **MAE: 0.8038**  | Está en el rango típico para filtrado colaborativo básico. Es más robusto que el RMSE.                                               |
| **Tiempo: 2.1s** | Lento para un modelo tan simple. Esto se debe a que calcular similitudes entre usuarios es costoso (requiere comparar muchos pares). |

**Conclusión**: UserKNN es fácil de entender, pero en datasets grandes sufre por escalabilidad y precisión.

---

## Análisis: **SVD**

| Aspecto          | Análisis                                                                                                     |
| ---------------- | ------------------------------------------------------------------------------------------------------------ |
| **RMSE: 0.9354** | Notablemente mejor que UserKNN. SVD aprende patrones latentes más eficientemente.                        |
| **MAE: 0.7376**  | También mejor que KNN. Muestra menor desviación promedio.                                                    |
| **Tiempo: 0.8s** | Mucho más rápido. SVD entrena con optimización por SGD y es más eficiente que comparar usuarios manualmente. |

**Conclusión**: SVD es más preciso y eficiente, aunque menos interpretable.

